In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud ,STOPWORDS
from PIL import Image
sample_submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
test = pd.read_csv("../input/nlp-getting-started/test.csv")
train = pd.read_csv("../input/nlp-getting-started/train.csv")

In [ ]:
train.head()

In [ ]:
train.columns[train.isnull().any()]

In [ ]:
train.info()

In [ ]:
numeric_cols = ['id']
categorical_cols = ['target']
object_cols = ['keyword','location','text']



************UNIVARIATE ANALYSIS************

**NUMERICAL COLUMNS**

In [ ]:
train['id'].unique()

In [ ]:
sns.distplot(train["id"])
train['id'].skew()

**CATEGORICAL COLUMNS**

In [ ]:
sns.countplot(train['target'])

****OBJECTIVE COLUMNS****

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
train_len=train[train['target']==1]['text'].str.len()
ax1.hist(train_len,color='red')
ax1.set_title('disaster tweets')
train_len=train[train['target']==0]['text'].str.len()
ax2.hist(train_len,color='green')
ax2.set_title('Not disaster tweets')
fig.suptitle('Characters in tweets')
plt.show()

In [ ]:
train['location'].unique()

In [ ]:
train[(train['location'].isnull())].shape

In [ ]:
train[(train['keyword'].isnull())].shape

In [ ]:
train.location.value_counts()

In [ ]:
train['keyword'].unique()

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
train.keyword.value_counts()

In [ ]:
train['text'].unique()

train['total_length'] = train['text'].apply(len)
train['capitals'] = train['text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
train['caps_vs_length'] = train.apply(lambda row: float(row['capitals'])/float(row['total_length']),axis=1)
train['num_exclamation_marks'] = train['text'].apply(lambda comment: comment.count('!'))
train['num_question_marks'] = train['text'].apply(lambda comment: comment.count('?'))
train['num_punctuation'] = train['text'].apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
train['num_symbols'] = train['text'].apply(lambda comment: sum(comment.count(w) for w in '*&$%'))
train['num_words'] = train['text'].apply(lambda comment: len(comment.split()))
train['num_unique_words'] = train['text'].apply(lambda comment: len(set(w for w in comment.split())))
train['words_vs_unique'] = train['num_unique_words'] / train['num_words']
train['num_smilies'] = train['text'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))

In [ ]:
train.head()

In [ ]:
train[train['target']==0]

In [ ]:
import re

**REMOVE THE http URL*

In [ ]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

In [ ]:
train['text']=train['text'].apply(lambda x : remove_URL(x))

**REMOVE THE html**

In [ ]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [ ]:
train['text']=train['text'].apply(lambda x : remove_html(x))

* Lower case*

In [ ]:
train['text'] = train['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train['text'].head()

*Removing Punctuation*

In [ ]:
train['text'] = train['text'].str.replace('[^\w\s]','')
train['text'].head()

*Removal of Stop Words*

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
train['text'] = train['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train['text'].head()

*Common word removal*

In [ ]:
dcw = pd.Series(' '.join(train['text']).split()).value_counts()[:20]
dcw

In [ ]:
dcw = list(dcw.index)
train['text'] = train['text'].apply(lambda x: " ".join(x for x in x.split() if x not in dcw))
train['text'].head()

*Rare words removal*

In [ ]:
freq = pd.Series(' '.join(train['text']).split()).value_counts()[-20:]
freq

In [ ]:
freq = list(freq.index)
train['text'] = train['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['text'].head()

*Spelling correction*

In [ ]:
#from textblob import TextBlob
#train['text'] = train['text'].apply(lambda x: str(TextBlob(x).correct()))

*stemming*

In [ ]:
from nltk.stem import PorterStemmer
st = PorterStemmer()
train['text']=train['text'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

*Lemmatization*

In [ ]:
from textblob import Word
train['text'] = train['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
train['text'].head()

In [ ]:
train.drop(columns=['location'],inplace=True)
train.drop(columns=['keyword'],inplace=True)
train.drop(columns=['id'],inplace=True)



In [ ]:
train.head()

In [ ]:
df_x=train['text']
df_y=train['target']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_x,df_y,test_size=0.3, random_state=9)
print(x_train.shape)
print(x_test.shape)

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
x_test.unique()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv= TfidfVectorizer(min_df=1,stop_words='english')

In [ ]:
X_train_counts=cv.fit_transform(x_train)

In [ ]:
X_train_counts.toarray()

In [ ]:
X_test_counts=cv.transform(x_test)

In [ ]:
cv.vocabulary_.get(u'algorithm')

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(X_train_counts,y_train)
clf.score(X_train_counts,y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test_counts)
accuracy_score(y_pred,y_test)

In [ ]:
from sklearn import metrics

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import  confusion_matrix
y_actu = pd.Series(y_test, name='Actual')
y_pred = pd.Series(y_pred, name='Predicted')
confusionMatrix = confusion_matrix(y_actu, y_pred)
print(confusionMatrix)


In [ ]:
for expected, predicted in zip(y_test, y_pred):
    if expected != predicted:
        print(expected, predicted)
        break

In [ ]:
docs_new=['its explosion happens on the second street']
X_new_counts=cv.transform(docs_new)
#X_new_tfidf=tfidf_transformer.fit_transform(X_new_counts)
predicted=clf.predict(X_new_counts)
predicted[0]


In [ ]:
for doc, target in zip(docs_new,predicted):
    print('%r => %s' % (doc, target))

In [ ]:
counter = 0
for expected, predicted in zip(y_test, y_pred):
    if expected != predicted:
        print(expected, predicted, counter)
        break
    counter  += 2

In [ ]:
x_test[6:7]